In [29]:
import torch
import numpy as np

z = torch.randn(4)

normal = torch.distributions.MultivariateNormal(torch.zeros(4), torch.eye(4))

normal.log_prob(z)

tensor(-6.2887)

In [34]:
np.sqrt((64 * 32 * 32)).astype(int)

256

In [35]:
import torch

# Create a tensor of shape (2, 3, 4)
x = torch.randn(2, 3, 4)

# Reshape the tensor to shape (2, 4, 3)
y = x.reshape(2, 4, 3)

# View and permute the tensor to shape (2, 4, 3)
z = x.view(2, 3, 4).permute(0, 2, 1)

# Check if y and z are equal
print(torch.allclose(y, z))

False


In [30]:
mean = z.mean()
var = z.var()

log_likelihood = (-4 / 2) * torch.log(2 * torch.tensor(np.pi)) - 0.5 * z.sum() ** 2

In [31]:
log_likelihood

tensor(-7.0451)